In [1]:
## Importables
from platform import python_version
print(f"Python version {python_version()}")

import sys
import os
import copy

#sys.path.append("/windir/c/Users/redas/Desktop/jupyter_directory/helpers/src/helpers/")

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA, NMF
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, normalize
from math import log10, log2, ceil, floor, sqrt

# And grab the helpers
import sys
sys.path.append("/windir/c/Users/redas/Desktop/jupyter_directory/helpers/src/helpers/")
from helpers import general_helpers as gh
from helpers import stats_helpers as sh
from helpers import mpl_plotting_helpers as mph
from helpers.proteomics_helpers import Peptide
from helpers import argcheck_helpers as ah

# Import scripts for processing PTM-SEA output
from py_scripts import generate_ssgsea_files as gsf
from py_scripts import managing_ssgsea_outputs as mso

from functions import *

# for the enrichment dotplots, will be moved eventually
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.patches import Circle

Python version 3.12.3
Loading the module: helpers.general_helpers

Loading the module: helpers.stats_helpers.py

numpy        2.2.5
scipy         1.15.2
pandas        2.2.3

Loading the module: helpers.mpl_plotting_helpers

Loading the module: helpers.argcheck_helpers

Loading the module: helpers.pandas_helpers

pandas        2.2.3
numpy         2.2.5

matplotlib    3.10.1
numpy         2.2.5

Python version 3.12.3


In [2]:
main_files = ["excel_sheet_input/20250411_jot1_ab_inhibitor_final.xls",
              "excel_sheet_input/20250411_jot1_t2kb_inhibitor_final.xls",
              "excel_sheet_input/20250425_car_raji_inhibitor_final.xls"]

ptm_files = ["figs/antibody/time_sea/output_combined_heatmap.txt",
             "figs/antibody/cont_sea/output_combined_heatmap.txt",
             "figs/pMHC/time_sea/output_combined_heatmap.txt",
             "figs/pMHC/cont_sea/output_combined_heatmap.txt",
             "figs/cd19/time_sea/output_combined_heatmap.txt",
             "figs/cd19/cont_sea/output_combined_heatmap.txt"]




gct_outnames =  ["figs/antibody/time_sea/qvalue.gct",
             "figs/antibody/cont_sea/qvalue.gct",
             "figs/pMHC/time_sea/qvalue.gct",
             "figs/pMHC/cont_sea/qvalue.gct",
             "figs/cd19/time_sea/qvalue.gct",
             "figs/cd19/cont_sea/qvalue.gct"]

reg_outnames = [ "figs/antibody/regression.pdf",
             "figs/pMHC/regression.pdf",
             "figs/cd19/regression.pdf"]

pca_outnames = ["figs/antibody/pca.pdf",
             "figs/pMHC/pca.pdf",
             "figs/cd19/pca.pdf"]

volc_outnames = ["figs/control_0_volcano.pdf",
                 "figs/control_150_volcano.pdf",
                 "figs/control_600_volcano.pdf",
                 "figs/stim_150s_volcano.pdf",
                 "figs/stim_600s_volcano.pdf",
                 "figs/control_ab_volcano.pdf",
                 "figs/control_pMHC_volcano.pdf",
                 "figs/control_cd19_volcano.pdf"]

site_outnames = ["figs/antibody/sites",
             "figs/pMHC/sites",
             "figs/cd19/sites"]

outdirs = ["figs/antibody/time_sea",
           "figs/antibody/cont_sea",
             "figs/pMHC/time_sea",
             "figs/pMHC/cont_sea",
             "figs/cd19/time_sea",
             "figs/cd19/cont_sea"]

exp_names = [r"J$^{OT1}$ (CD3/CD28)", r"J$^{OT1}$ (pT2-K$^{b})$", r"CD19-$28\zeta$-CAR (Raji)"]

xaxis_strs = ["2.5 min vs 0 min",
              "10 min vs 0 min"]

yax_strs = [r"$-\log_{10}(q)$",
              r"$-\log_{10}(q)$",
              r"$-\log_{10}(q)$"]

yax2_strs = [r"DMSO", r"RDN 2150", r"Soquelitinib"]

columns = ["UNIPROT Gene Name",                                #0
           "phosphosite annotated",                            #1
            "peakarea manual 1 rep1 thresholded timepoint1",   #2
            "peakarea manual 1 rep2 thresholded timepoint1",   #3
            "peakarea manual 1 rep3 thresholded timepoint1",   #4
            "peakarea manual 1 rep4 thresholded timepoint1",   #5
            "peakarea manual 1 rep5 thresholded timepoint1",   #5
            "peakarea manual 1 rep1 thresholded timepoint2",   #6
            "peakarea manual 1 rep2 thresholded timepoint2",   #7
            "peakarea manual 1 rep3 thresholded timepoint2",   #8
            "peakarea manual 1 rep4 thresholded timepoint2",   #9
            "peakarea manual 1 rep5 thresholded timepoint2",   #9
            "peakarea manual 1 rep1 thresholded timepoint3",   #10
            "peakarea manual 1 rep2 thresholded timepoint3",   #11
            "peakarea manual 1 rep3 thresholded timepoint3",   #12
            "peakarea manual 1 rep4 thresholded timepoint3",   #13
            "peakarea manual 1 rep5 thresholded timepoint3",   #14
            "peakarea manual 1 rep1 thresholded timepoint4",   #15
            "peakarea manual 1 rep2 thresholded timepoint4",   #16
            "peakarea manual 1 rep3 thresholded timepoint4",   #17
            "peakarea manual 1 rep4 thresholded timepoint4",   #18
            "peakarea manual 1 rep5 thresholded timepoint4",   #19
            "peakarea manual 1 rep1 thresholded timepoint5",   #20
            "peakarea manual 1 rep2 thresholded timepoint5",   #21
            "peakarea manual 1 rep3 thresholded timepoint5",   #22
            "peakarea manual 1 rep4 thresholded timepoint5",   #23
            "peakarea manual 1 rep5 thresholded timepoint5",   #24
            "peakarea manual 1 rep1 thresholded timepoint6",   #25
            "peakarea manual 1 rep2 thresholded timepoint6",   #26
            "peakarea manual 1 rep3 thresholded timepoint6",   #27
            "peakarea manual 1 rep4 thresholded timepoint6",   #28
            "peakarea manual 1 rep5 thresholded timepoint6",   #29
            "peakarea manual 1 rep1 thresholded timepoint7",   #30
            "peakarea manual 1 rep2 thresholded timepoint7",   #31
            "peakarea manual 1 rep3 thresholded timepoint7",   #32
            "peakarea manual 1 rep4 thresholded timepoint7",   #33
            "peakarea manual 1 rep5 thresholded timepoint7",   #34
            "peakarea manual 1 rep1 thresholded timepoint8",   #35
            "peakarea manual 1 rep2 thresholded timepoint8",   #36
            "peakarea manual 1 rep3 thresholded timepoint8",   #37
            "peakarea manual 1 rep4 thresholded timepoint8",   #38
            "peakarea manual 1 rep5 thresholded timepoint8",   #39
            "peakarea manual 1 rep1 thresholded timepoint9",   #40
            "peakarea manual 1 rep2 thresholded timepoint9",   #41
            "peakarea manual 1 rep3 thresholded timepoint9",   #42
            "peakarea manual 1 rep4 thresholded timepoint9",   #43
            "peakarea manual 1 rep5 thresholded timepoint9",   #44
            "qvalues for SILAC timepoint1",                    #45
            "qvalues for SILAC timepoint2",                    #46
            "qvalues for SILAC timepoint3",                    #47
            "qvalues for SILAC timepoint4",                    #48
            "qvalues for SILAC timepoint5",                    #49
            "qvalues for SILAC timepoint6",                    #50
            "qvalues for SILAC timepoint7",                    #51
            "qvalues for SILAC timepoint8",                    #52
            "qvalues for SILAC timepoint9",                    #53
            "qvalues for SILAC timepoint10",                    #54
            "qvalues for SILAC timepoint11",                    #55
            "qvalues for SILAC timepoint12",                    #56
            "assigned sequence",                               #57
            "Kegg unique index",                               #58
            "peptide sequence GCT format centered on 1st site",#59
            "peptide sequence GCT format centered on 2nd site",#60
            "peptide sequence GCT format centered on 3rd site" #61
           ]

fcond_cols = ["UNIPROT Gene Name",                                #0
           "phosphosite annotated",                            #1
            "peakarea manual 1 rep1 thresholded timepoint1",   #2
            "peakarea manual 1 rep2 thresholded timepoint1",   #3
            "peakarea manual 1 rep3 thresholded timepoint1",   #4
            "peakarea manual 1 rep4 thresholded timepoint1",   #5
            "peakarea manual 1 rep5 thresholded timepoint1",   #5
            "peakarea manual 1 rep1 thresholded timepoint2",   #6
            "peakarea manual 1 rep2 thresholded timepoint2",   #7
            "peakarea manual 1 rep3 thresholded timepoint2",   #8
            "peakarea manual 1 rep4 thresholded timepoint2",   #9
            "peakarea manual 1 rep5 thresholded timepoint2",   #9
            "peakarea manual 1 rep1 thresholded timepoint3",   #10
            "peakarea manual 1 rep2 thresholded timepoint3",   #11
            "peakarea manual 1 rep3 thresholded timepoint3",   #12
            "peakarea manual 1 rep4 thresholded timepoint3",   #13
            "peakarea manual 1 rep5 thresholded timepoint3",   #14
            "peakarea manual 1 rep1 thresholded timepoint4",   #15
            "peakarea manual 1 rep2 thresholded timepoint4",   #16
            "peakarea manual 1 rep3 thresholded timepoint4",   #17
            "peakarea manual 1 rep4 thresholded timepoint4",   #18
            "peakarea manual 1 rep5 thresholded timepoint4",   #19
            "peakarea manual 1 rep1 thresholded timepoint5",   #20
            "peakarea manual 1 rep2 thresholded timepoint5",   #21
            "peakarea manual 1 rep3 thresholded timepoint5",   #22
            "peakarea manual 1 rep4 thresholded timepoint5",   #23
            "peakarea manual 1 rep5 thresholded timepoint5",   #24
            "peakarea manual 1 rep1 thresholded timepoint6",   #25
            "peakarea manual 1 rep2 thresholded timepoint6",   #26
            "peakarea manual 1 rep3 thresholded timepoint6",   #27
            "peakarea manual 1 rep4 thresholded timepoint6",   #28
            "peakarea manual 1 rep5 thresholded timepoint6",   #29
            "peakarea manual 1 rep1 thresholded timepoint7",   #30
            "peakarea manual 1 rep2 thresholded timepoint7",   #31
            "peakarea manual 1 rep3 thresholded timepoint7",   #32
            "peakarea manual 1 rep4 thresholded timepoint7",   #33
            "peakarea manual 1 rep5 thresholded timepoint7",   #34
            "peakarea manual 1 rep1 thresholded timepoint8",   #35
            "peakarea manual 1 rep2 thresholded timepoint8",   #36
            "peakarea manual 1 rep3 thresholded timepoint8",   #37
            "peakarea manual 1 rep4 thresholded timepoint8",   #38
            "peakarea manual 1 rep5 thresholded timepoint8",   #39
            "peakarea manual 1 rep1 thresholded timepoint9",   #40
            "peakarea manual 1 rep2 thresholded timepoint9",   #41
            "peakarea manual 1 rep3 thresholded timepoint9",   #42
            "peakarea manual 1 rep4 thresholded timepoint9",   #43
            "peakarea manual 1 rep5 thresholded timepoint9",   #44
            "qvalues for SILAC timepoint1",                    #45
            "qvalues for SILAC timepoint2",                    #46
            "qvalues for SILAC timepoint3",                    #47
            "qvalues for SILAC timepoint4",                    #48
            "qvalues for SILAC timepoint5",                    #49
            "qvalues for SILAC timepoint6",                    #50
            "qvalues for SILAC timepoint7",                    #51
            "qvalues for SILAC timepoint8",                    #52
            "qvalues for SILAC timepoint9",                    #53
            "qvalues for SILAC timepoint10",                    #54
            "qvalues for SILAC timepoint11",                    #55
            "qvalues for SILAC timepoint12",                    #56
            "assigned sequence",                               #57
            "Kegg unique index",                               #58
            "flank1"                                           #59
           ]

newcol = ["Gene",
          "Phosphorylation Site", 
          "D 0s R1", "D 0s R2", "D 0s R3", "D 0s R4", "D 0s R5",
          "D 150s R1", "D 150s R2", "D 150s R3", "D 150s R4", "D 150s R5",
          "D 600s R1", "D 600s R2", "D 600s R3", "D 600s R4", "D 600s R5",
          "R 0s R1", "R 0s R2", "R 0s R3", "R 0s R4", "R 0s R5",
          "R 150s R1", "R 150s R2", "R 150s R3", "R 150s R4", "R 150s R5",
          "R 600s R1", "R 600s R2", "R 600s R3", "R 600s R4", "R 600s R5",
          "S 0s R1", "S 0s R2", "S 0s R3", "S 0s R4", "S 0s R5",
          "S 150s R1", "S 150s R2", "S 150s R3", "S 150s R4", "S 150s R5",
          "S 600s R1", "S 600s R2", "S 600s R3", "S 600s R4", "S 600s R5",
          "D 150s vs D 0s qvalue", "D 600s vs D 0s qvalue", 
          "R 150s vs R 0s qvalue", "R 600s vs R 0s qvalue", 
          "S 150s vs S 0s qvalue", "S 600s vs S 0s qvalue", 
          "R 0s vs D 0s qvalue", "R 150s vs D 150s qvalue", "R 600s vs D 600s qvalue",
          "S 0s vs D 0s qvalue", "S 150s vs D 150s qvalue", "S 600s vs D 600s qvalue",
          "Sequence",
          "KEGG", "Flank 1"]

sample_groups = ["D 0s", "D 150s", "D 600s", 
                 "R 0s", "R 150s", "R 600s",
                 "S 0s", "S 150s", "S 600s"]

comparisons = [("D 150s", "D 0s"), ("D 600s", "D 0s"),
              ("R 150s", "R 0s"), ("R 600s", "R 0s"),
              ("S 150s", "S 0s"), ("S 600s", "S 0s"),
              ("R 0s", "D 0s"), ("R 150s", "D 150s"), ("R 600s", "D 600s"),
              ("S 0s", "D 0s"), ("S 150s", "D 150s"), ("S 600s", "D 600s")]

comp_strs = [f"{c[0]} vs {c[1]}" for c in comparisons]

colours = [mph.handle_colours("pinks", 3),   
           mph.handle_colours("blues", 3),   
           mph.handle_colours("purples", 3)]

# Variables needed for volcano plots
sigs = [0.0005,0.005,0.05] 
rightlabs = ["2.5 min vs 0 min", "10 min vs 0m"]
leftlabs = [fr"$-\log_{{10}}(q)$" for i in range(2)]
bottomlabs = [fr"$\log_{{2}}(5m)- \log_{{2}}({{0m}})$" for i in range(5)]

#slices = [[slice(0,5), slice(5,10), slice(10,15)],
#          [slice(0,5), slice(5,10), slice(10,15)],
#          [slice(0,5), slice(5,10), slice(10,14)],
#          [slice(0,5), slice(5,10), slice(10,15)],
#          [slice(0,5), slice(5,10), slice(10,14)],]

def remove_missing_cols(a_matrix,
                        transpose = True,
                        remove = True):
    """
    Given a matrix, mark and remove any columns that are entirely missing,
    then return the update matrix.

    Assumes the matrix is a list of same size lists, with headers.
    """
    if transpose:
        t_mat = gh.transpose(*a_matrix)
    else:
        t_mat = a_matrix.copy()
    # Find any columns with only the header value
    missing_inds = [i for i in range(len(t_mat)) if sum([1 for _ in t_mat[i] if _ == _]) == 1]
    # Remove them if requested
    if remove:
        t_mat = [t_mat[i] for i in range(len(t_mat)) if i not in missing_inds]
    if transpose:
        return gh.transpose(*t_mat)
    else:
        return t_mat

def get_index_groups(matrix_heads,
                     core_group_strs, 
                    retain_group = True):
    """
    First, parse all of the heads into groups, then index the original list with them

    core_group_strs needs to have unique substrings of the heads for this to work
    """
    # First, group the headers
    groups = [(cgs,[h for h in matrix_heads if cgs in h])
              for cgs in core_group_strs]
    if retain_group:
        inds = {subgroup[0] : [matrix_heads.index(h) for h in subgroup[1]]
                for subgroup in groups}
    else:
        inds = [[matrix_heads.index(h) for h in subgroup[1]]
                for subgroup in groups]
    return inds

def calc_fc(g1, g2, log2_trans = True):
    # Assumes prefiltering of nans
    if log2_trans:
        g1 = [log2(val) for val in g1]
        g2 = [log2(val) for val in g2]
    av1 = sh.mean(g1, threshold = 1)
    av2 = sh.mean(g2, threshold = 1)
    if log2_trans:
        return av1-av2
    else:
        return av1/av2

def safe_log2(a_number):
    try:
        return log2(a_number)
    except:
        print(a_number)
        return float("nan")

In [3]:
fc_heads = [f"{c[0]} vs {c[1]} Fold-change" for c in comparisons]

## Grab the excel files and manage the data, including flanking sequences
file_dfs = [read_and_filter(f, columns) for f in main_files]
file_lists = [[list(row) for row in list(df.to_numpy())] for df in file_dfs]
file_lists = [ [newcol + ['U_ID', 'q_num','min_q', 'max_fc'] + fc_heads] + dup_flanks_matrix(f[1:], [61,62,63]) for f in file_lists]
file_lists = [[row for row in f if row[61] == row[61]] for f in file_lists]

######## Now, since the FCs were bugged, calcultate the desired FCs.
# First, grab the group indices of missing cols
group_inds = get_index_groups(newcol, sample_groups)
# Then we have to make all the FCs and add them to the lists
for i in range(len(file_lists)):
    for j in range(len(file_lists[i])):
        for c in comparisons:
            if j != 0:
                file_lists[i][j].append(calc_fc([file_lists[i][j][k] for k in group_inds[c[0]] if file_lists[i][j][k] == file_lists[i][j][k]],
                                           [file_lists[i][j][k] for k in group_inds[c[1]] if file_lists[i][j][k] == file_lists[i][j][k]],
                                           log2_trans = False))
            else:
                pass

###### Now we should grab the q-value/FC pairs
# Use the comp_strs list to find the heads with substrings
fq_pairs = [(c,[item for item in file_lists[0][0] if c in item and any(["qvalue" in item, "Fold-change" in item])])
         for c in comp_strs]
# and index those positions
fq_pair_inds = [(pair[0],[file_lists[0][0].index(head) for head in pair[1]])
                for pair in fq_pairs]

file_lists = [[row + [ptmsea_transform(row[p[1][0]], row[p[1][1]], f"{p[0]} PTMSEA") for p in fq_pair_inds] for row in f]
             for f in file_lists]

file_lists = [[f[0]] + sorted(f[1:], key = lambda x: x[61]) for f in file_lists]
gct_inds = [[f"{row[61]}-p" for row in f[1:]] for f in file_lists]
gct_timecourse = [[row[-12:-6] for row in f[1:]] for f in file_lists]
gct_timecourse_heads = [string.replace(" ", "_") for string in comp_strs[:6]]
gct_control = gct_vals = [[row[-6:] for row in f[1:]] for f in file_lists]
gct_control_heads = [string.replace(" ", "_") for string in comp_strs[6:]]

# Finally, write the GCT files for PTM-SEA later
for i in range(len(file_lists)):
    write_gct_file(gct_inds[i], gct_timecourse[i], gct_timecourse_heads, gct_outnames[2*i])
    write_gct_file(gct_inds[i], gct_control[i], gct_control_heads, gct_outnames[2*i+1])

In [4]:
for i in range(len(file_lists[0][0])):
    print(f"{i} {file_lists[0][0][i]} | {file_lists[0][69][i]}")

0 Gene | PCBP2
1 Phosphorylation Site | Y201
2 D 0s R1 | 9653850.0
3 D 0s R2 | 8805801.0
4 D 0s R3 | 3721802.0
5 D 0s R4 | 9652656.0
6 D 0s R5 | 13553537.0
7 D 150s R1 | 31379796.0
8 D 150s R2 | 6466997.0
9 D 150s R3 | 13972129.0
10 D 150s R4 | 29218529.0
11 D 150s R5 | 24594654.0
12 D 600s R1 | 8682102.0
13 D 600s R2 | 9995246.0
14 D 600s R3 | 4738367.0
15 D 600s R4 | 6623532.0
16 D 600s R5 | 5149703.0
17 R 0s R1 | 5297067.0
18 R 0s R2 | 5274633.0
19 R 0s R3 | 5037227.0
20 R 0s R4 | 1905388.0
21 R 0s R5 | 2720360.0
22 R 150s R1 | 17521057.0
23 R 150s R2 | 14972374.0
24 R 150s R3 | 18739259.0
25 R 150s R4 | 14698631.0
26 R 150s R5 | 12263453.0
27 R 600s R1 | 3618404.0
28 R 600s R2 | 4671096.0
29 R 600s R3 | 6349256.0
30 R 600s R4 | 6014860.0
31 R 600s R5 | 5824657.0
32 S 0s R1 | 2877801.0
33 S 0s R2 | 6515441.0
34 S 0s R3 | 5071940.0
35 S 0s R4 | nan
36 S 0s R5 | nan
37 S 150s R1 | 13934891.0
38 S 150s R2 | 12203952.0
39 S 150s R3 | 15055051.0
40 S 150s R4 | 8484894.0
41 S 150s R5 | 17

In [5]:
# Use the comp_strs list to find the heads with substrings
samps = [(c,[item for item in file_lists[0][0] if c in item and not any(["qvalue" in item, "Fold-change" in item, "PTMSEA" in item])])
         for c in sample_groups]
# and index those positions
samp_inds = [(pair[0],[file_lists[0][0].index(head) for head in pair[1]])
                for pair in samps]

# Becuase there's an uneven number of reps, this just fails and it's not worth it imo
narnar = [multi_reg_lineplot(file_dfs[i], 
                             groups = reg_gs, 
                             labels = ["DMSO 0m", "DMSO 2.5m", "DMSO 10m", 
                                      "RDN 0m", "RDN 2.5m", "RDN 10m", 
                                      "Soq 0m", "Soq 2.5m", "Soq 10m", ],
                             savefile = reg_outnames[i]) for i in range(len(file_dfs))]
unpacked_c = gh.unpack_list(colours)
# Perform PCA clustering using the file dataframes.

#### Split each dataframe into D, R, S, and do them separately.
#### Too much data for one PCA plot
pca_ax = cluster_plotting(file_dfs, # list with minimum 1 df
                 reg_gs,     
                 exp_names[0],
                 pca_outnames,
                 [p[1] for p in samp_inds],
                 ["DMSO 0m", "DMSO 2.5m", "DMSO 10m", 
                    "RDN 0m", "RDN 2.5m", "RDN 10m", 
                    "Soq 0m", "Soq 2.5m", "Soq 10m", ],
                 [unpacked_c, unpacked_c, unpacked_c],
                 markers = ["o", "8", "h", 
                            "^", "<", ">",
                            "s", "D", "d"],
                 cluster = 'PCA',
                 markersize=75,
                 square = False,
                forced_axes = [[(-15, 20), (-10,20)],
                               [(-15,20), (-15,10)],
                               [(-10,15),(-10,20)]],
                          textdict = dict(fontfamily = "sans-serif",
                 font = "Arial",
                 fontweight = "bold",
                 fontsize = 14),
                 pca_kwargs = dict(n_components = 2,
                                   whiten = False,
                                   svd_solver = "full",
                                   tol = 0))

[6]
[3, 4]
[556, 852, 661, 517, 707, 590, 581, 800, 551]
[3]
[4]
[296, 313, 339, 259, 269, 287, 319, 354, 348]
[]
[275, 255, 274, 279, 261, 264, 242, 296, 297]
[(-15, 20), (-10, 20)]
[(-15, 20), (-15, 10)]
[(-10, 15), (-10, 20)]


In [ ]:
# Want to get all the q-values groups nicely for plotting 

file_lists_t = [gh.transpose(*f) for f in file_lists]

control_ab_qs = []
control_ab_fc = []

control_mhc_qs = []
control_mhc_fc = []

control_car_qs = []
control_car_fc = []

control_0_qs = []
control_0_fc = []

control_150_qs = []
control_150_fc = []

control_600_qs = []
control_600_fc = []

time_150_qs = []
time_150_fc = []

time_600_qs = []
time_600_fc = []

counter = 0



for f in file_lists_t:
    time_150_qs += [[f[47][1:], f[49][1:], f[51][1:]]]
    time_600_qs += [[f[48][1:], f[50][1:], f[52][1:]]]
    control_0_qs += [[f[53][1:], f[56][1:]]]
    control_150_qs += [[f[54][1:], f[57][1:]]]
    control_600_qs += [[f[55][1:], f[58][1:]]]

    time_150_fc += [[f[66][1:], f[68][1:], f[70][1:]]]
    time_600_fc += [[f[67][1:], f[69][1:], f[71][1:]]]
    control_0_fc += [[f[72][1:], f[75][1:]]]
    control_150_fc += [[f[73][1:], f[76][1:]]]
    control_600_fc += [[f[74][1:], f[77][1:]]]
    for j in range(3):
        cont_qs = [f[53+j][1:], f[56+j][1:]]
        cont_fc = [f[72+j][1:], f[75+j][1:]]
        if counter == 0:
            control_ab_qs += [cont_qs]
            control_ab_fc += [cont_fc]
        elif counter == 1:
            control_mhc_qs += [cont_qs]
            control_mhc_fc += [cont_fc]
        else:
            control_car_qs += [cont_qs]
            control_car_fc += [cont_fc]
    counter +=1

In [ ]:
len(control_ab_qs[0])

In [ ]:
mph.volcano_array(control_0_qs, 
                  control_0_fc,
                   xlim = 10, ylim = 7,
                   fc_transform = "log2",
                   sig_transform = "log10",
                   fc_label_transform = "none",
                  left_labels = yax_strs,
                  top_labels = yax2_strs[1:],
                  right_labels = exp_names,
                  colours =  [colours[1:], colours[1:], colours[1:]],
                   sig_cutoffs = sigs,
                   fc_cutoff = 1,
                   save = volc_outnames[0])

mph.volcano_array(control_150_qs, 
                  control_150_fc,
                   xlim = 10, ylim = 7,
                   fc_transform = "log2",
                   sig_transform = "log10",
                   fc_label_transform = "none",
                  left_labels = yax_strs,
                  top_labels = yax2_strs[1:],
                  right_labels = exp_names,
                  colours =  [colours[1:], colours[1:], colours[1:]],
                   sig_cutoffs = sigs,
                   fc_cutoff = 1,
                   save = volc_outnames[1])

mph.volcano_array(control_600_qs, 
                  control_600_fc,
                   xlim = 10, ylim = 7,
                   fc_transform = "log2",
                   sig_transform = "log10",
                   fc_label_transform = "none",
                  left_labels = yax_strs,
                  top_labels = yax2_strs[1:],
                  right_labels = exp_names,
                  colours =  [colours[1:], colours[1:], colours[1:]],
                   sig_cutoffs = sigs,
                   fc_cutoff = 1,
                   save = volc_outnames[2])


mph.volcano_array(time_150_qs, 
                  time_150_fc,
                   xlim = 10, ylim = 7,
                   fc_transform = "log2",
                   sig_transform = "log10",
                   fc_label_transform = "none",
                  left_labels = yax_strs,
                  top_labels = yax2_strs,
                  right_labels = exp_names,
                  colours =  [colours, colours, colours],
                   sig_cutoffs = sigs,
                   fc_cutoff = 1,
                   save = volc_outnames[3])

mph.volcano_array(time_600_qs, 
                  time_600_fc,
                   xlim = 10, ylim = 7,
                   fc_transform = "log2",
                   sig_transform = "log10",
                   fc_label_transform = "none",
                  left_labels = yax_strs,
                  top_labels = yax2_strs,
                  right_labels = exp_names,
                  colours =  [colours, colours, colours],
                   sig_cutoffs = sigs,
                   fc_cutoff = 1,
                   save = volc_outnames[4])



In [ ]:
mph.volcano_array(control_ab_qs, 
                  control_ab_fc,
                   xlim = 10, ylim = 7,
                   fc_transform = "log2",
                   sig_transform = "log10",
                   fc_label_transform = "none",
                  left_labels = yax_strs,
                  top_labels = yax2_strs[1:],
                  right_labels = ["0 min", "2.5 min", "10 min"],
                  colours =  [colours[1:], colours[1:], colours[1:]],
                   sig_cutoffs = sigs,
                   fc_cutoff = 1,
                   save = volc_outnames[5])

mph.volcano_array(control_mhc_qs, 
                  control_mhc_fc,
                   xlim = 10, ylim = 7,
                   fc_transform = "log2",
                   sig_transform = "log10",
                   fc_label_transform = "none",
                  left_labels = yax_strs,
                  top_labels = yax2_strs[1:],
                  right_labels = ["0 min", "2.5 min", "10 min"],
                  colours =  [colours[1:], colours[1:], colours[1:]],
                   sig_cutoffs = sigs,
                   fc_cutoff = 1,
                   save = volc_outnames[6])

mph.volcano_array(control_car_qs, 
                  control_car_fc,
                   xlim = 10, ylim = 7,
                   fc_transform = "log2",
                   sig_transform = "log10",
                   fc_label_transform = "none",
                  left_labels = yax_strs,
                  top_labels = yax2_strs[1:],
                  right_labels = ["0 min", "2.5 min", "10 min"],
                  colours =  [colours[1:], colours[1:], colours[1:]],
                   sig_cutoffs = sigs,
                   fc_cutoff = 1,
                   save = volc_outnames[7])


In [ ]:
file_lists[0][1][0]

In [ ]:
qcol_inds = [i for i in range(len(file_lists[0][0])) if "qvalue" in file_lists[0][0][i]]
fc_inds = [i for i in range(len(file_lists[0][0])) if "Fold-change" in file_lists[0][0][i]]
samp_inds = [i for i in range(len(file_lists[0][0])) if "s R" in file_lists[0][0][i]] 

time_150_inds = [[47,49,51],[66,68,70]]
time_600_inds = [[48,50,52],[67,69,71]]
control_r_inds = [[53,54,55],[72,73,74]]
control_s_inds = [[56,57,58],[75,76,77]]

In [6]:
file_lists[0][0]

['Gene',
 'Phosphorylation Site',
 'D 0s R1',
 'D 0s R2',
 'D 0s R3',
 'D 0s R4',
 'D 0s R5',
 'D 150s R1',
 'D 150s R2',
 'D 150s R3',
 'D 150s R4',
 'D 150s R5',
 'D 600s R1',
 'D 600s R2',
 'D 600s R3',
 'D 600s R4',
 'D 600s R5',
 'R 0s R1',
 'R 0s R2',
 'R 0s R3',
 'R 0s R4',
 'R 0s R5',
 'R 150s R1',
 'R 150s R2',
 'R 150s R3',
 'R 150s R4',
 'R 150s R5',
 'R 600s R1',
 'R 600s R2',
 'R 600s R3',
 'R 600s R4',
 'R 600s R5',
 'S 0s R1',
 'S 0s R2',
 'S 0s R3',
 'S 0s R4',
 'S 0s R5',
 'S 150s R1',
 'S 150s R2',
 'S 150s R3',
 'S 150s R4',
 'S 150s R5',
 'S 600s R1',
 'S 600s R2',
 'S 600s R3',
 'S 600s R4',
 'S 600s R5',
 'D 150s vs D 0s qvalue',
 'D 600s vs D 0s qvalue',
 'R 150s vs R 0s qvalue',
 'R 600s vs R 0s qvalue',
 'S 150s vs S 0s qvalue',
 'S 600s vs S 0s qvalue',
 'R 0s vs D 0s qvalue',
 'R 150s vs D 150s qvalue',
 'R 600s vs D 600s qvalue',
 'S 0s vs D 0s qvalue',
 'S 150s vs D 150s qvalue',
 'S 600s vs D 600s qvalue',
 'Sequence',
 'KEGG',
 'Flank 1',
 'U_ID',
 'q_num

In [ ]:
time_150_peps = []

for f in file_lists:
    file_peps = []
    for row in f[1:]:
        if row[0] == row[0]:
            file_peps.append(Peptide([row[i] for i in samp_inds],                   # Sample measurements
                                 [file_lists[0][0][q] for q in samp_inds],    # Sample heads
                                 ["D 0s", "D 150s", "D 600s",
                                  "R 0s", "R 150s", "R 600s",
                                  "S 0s", "S 150s", "S 600s"],                  # Group labels
                                 row[59],                                       # Assigned sequence
                                 statistics = [row[i] for i in time_150_inds[0]],                   # Statistical values
                                 statistics_headers = ["D 150s vs D 0s qvalue",
                                                       "R 150s vs R 0s qvalue",
                                                       "S 150s vs S 0s qvalue"],      # qvalue headers
                                 foldchange = [safe_log2(row[i]) for i in time_150_inds[1]],                   # Foldchange values
                                 foldchange_headers = ["D 150s vs D 0s",
                                                       "R 150s vs R 0s",
                                                       "S 150s vs S 0s"],
                                 sites = fr"$^{{{row[1]}}}$",                   # Site position
                                 gene = row[0],                                 # Gene name
                                 unique_id = row[62],
                                 colours = colours,
                                 markers = ["o", "o", "o",
                                            "^", "^", "^",
                                            "s", "s", "s"],))
    time_150_peps.append(file_peps)

i=0
for peps in time_150_peps:
    print(site_outnames[i])
    make_all_pepplots(peps, path = site_outnames[i],
                  subset = ["all"], exclude = ["none"],
                  comparisons = ["D 150s vs D 0s qvalue",
                                 "R 150s vs R 0s qvalue",
                                 "S 150s vs S 0s qvalue"],
                  foldchange_group = " 0s",
                      global_max = 6,
                      heatname_tag = "150s",
                      heatmap_kwargs = {'aspect': 'equal', 'remove_spines': False, 'subplot_args': {'figsize': (14, 1)}, 
                                        'colorbar_args': {'orientation': 'vertical', 'location': 'right', 'shrink': 2}, 
                                        'textdict': {'fontfamily': 'sans-serif', 'font': 'Arial', 'fontweight': 'bold'}, 
                                        'img_name': 'figs/pep_plots/tbc1d5/heatmaps/tbc1d5_2_foldchange_all', 
                                        'heat_title': '', 
                                        'clb_label': 'log$_{2}$(FC)',
                                        'maxs': [-6, 6], 
                                        "sig_bounds" : [0.05,0.005,0.0005],
                                        'cmap': mph.trans})
    i+=1


In [ ]:
time_600_peps = []

for f in file_lists:
    file_peps = []
    for row in f[1:]:
        if row[0] == row[0]:
            file_peps.append(Peptide([row[i] for i in samp_inds],                   # Sample measurements
                                 [file_lists[0][0][q] for q in samp_inds],    # Sample heads
                                 ["D 0s", "D 150s", "D 600s",
                                  "R 0s", "R 150s", "R 600s",
                                  "S 0s", "S 150s", "S 600s"],                  # Group labels
                                 row[59],                                       # Assigned sequence
                                 statistics = [row[i] for i in time_600_inds[0]],                   # Statistical values
                                 statistics_headers = ["D 600s vs D 0s qvalue",
                                                       "R 600s vs R 0s qvalue",
                                                       "S 600s vs S 0s qvalue"],      # qvalue headers
                                 foldchange = [safe_log2(row[i]) for i in time_600_inds[1]],                   # Foldchange values
                                 foldchange_headers = ["D 600s vs D 0s",
                                                       "R 600s vs R 0s",
                                                       "S 600s vs S 0s"],
                                 sites = fr"$^{{{row[1]}}}$",                   # Site position
                                 gene = row[0],                                 # Gene name
                                 unique_id = row[62],
                                 colours = colours,
                                 markers = ["o", "o", "o",
                                            "^", "^", "^",
                                            "s", "s", "s"],))
    time_600_peps.append(file_peps)

i=0
for peps in time_600_peps:
    print(site_outnames[i])
    make_all_pepplots(peps, path = site_outnames[i],
                  subset = ["all"], exclude = ["none"],
                  comparisons = ["D 150s vs D 0s qvalue",
                                 "R 150s vs R 0s qvalue",
                                 "S 150s vs S 0s qvalue"],
                  foldchange_group = " 0s",
                      global_max = 6,
                      heatname_tag = "600s",
                      heatmap_kwargs = {'aspect': 'equal', 'remove_spines': False, 'subplot_args': {'figsize': (14, 1)}, 
                                        'colorbar_args': {'orientation': 'vertical', 'location': 'right', 'shrink': 2}, 
                                        'textdict': {'fontfamily': 'sans-serif', 'font': 'Arial', 'fontweight': 'bold'}, 
                                        'img_name': 'figs/pep_plots/tbc1d5/heatmaps/tbc1d5_2_foldchange_all', 
                                        'heat_title': '', 
                                        'clb_label': 'log$_{2}$(FC)',
                                        'maxs': [-6, 6], 
                                        "sig_bounds" : [0.05,0.005,0.0005],
                                        'cmap': mph.trans})
    i+=1


In [ ]:
cont_peps = []

for f in file_lists:
    file_peps = []
    for row in f[1:]:
        if row[0] == row[0]:
            file_peps.append(Peptide([row[i] for i in samp_inds],                   # Sample measurements
                                 [file_lists[0][0][q] for q in samp_inds],    # Sample heads
                                 ["D 0s", "D 150s", "D 600s",
                                  "R 0s", "R 150s", "R 600s",
                                  "S 0s", "S 150s", "S 600s"],                  # Group labels
                                 row[59],                                       # Assigned sequence
                                 statistics = [row[i] for i in control_r_inds[0] + control_s_inds[0]],                   # Statistical values
                                 statistics_headers = ["R 0s vs D 0s qvalue",
                                                       "R 150s vs D 150s qvalue",
                                                       "R 600s vs D 600s qvalue",
                                                       "S 0s vs S 0s qvalue",
                                                       "S 150s vs S 150s qvalue",
                                                       "S 600s vs S 600s qvalue",],      # qvalue headers
                                 foldchange = [safe_log2(row[i]) for i in control_r_inds[1] + control_s_inds[1]],                   # Foldchange values
                                 foldchange_headers = ["R 0s vs D 0s",
                                                       "R 150s vs D 150s",
                                                       "R 600s vs D 600s",
                                                       "S 0s vs S 0s",
                                                       "S 150s vs S 150s",
                                                       "S 600s vs S 600s",],
                                 sites = fr"$^{{{row[1]}}}$",                   # Site position
                                 gene = row[0],                                 # Gene name
                                 unique_id = row[62],
                                 colours = colours,
                                 markers = ["o", "o", "o",
                                            "^", "^", "^",
                                            "s", "s", "s"],))
    cont_peps.append(file_peps)

control_tags = ["ab", "mhc", "car"]

i=0
for peps in cont_peps:
    print(site_outnames[i])
    make_all_pepplots(peps, path = site_outnames[i],
                  subset = ["all"], exclude = ["none"],
                  comparisons = ["D 150s vs D 0s qvalue",
                                 "R 150s vs R 0s qvalue",
                                 "S 150s vs S 0s qvalue"],
                  foldchange_group = "D 0s",
                      global_max = 6,
                      heatname_tag = control_tags[i],
                      heatmap_kwargs = {'aspect': 'equal', 'remove_spines': False, 'subplot_args': {'figsize': (14, 1)}, 
                                        'colorbar_args': {'orientation': 'vertical', 'location': 'right', 'shrink': 2}, 
                                        'textdict': {'fontfamily': 'sans-serif', 'font': 'Arial', 'fontweight': 'bold'}, 
                                        'img_name': 'figs/pep_plots/tbc1d5/heatmaps/tbc1d5_2_foldchange_all', 
                                        'heat_title': '', 
                                        'clb_label': 'log$_{2}$(FC)',
                                        'maxs': [-6, 6], 
                                        "sig_bounds" : [0.05,0.005,0.0005],
                                        'cmap': mph.trans})
    i+=1


In [ ]:
gsf.imp_main("/mnt/c/Users/redas/Desktop/jupyter_directory/salomon_lab_folder/rdn_soq_project/proteomics",
             "qvalue",
             "/mnt/c/Users/redas/Desktop/jupyter_directory/salomon_lab_folder/rdn_soq_project/proteomics/r_scripts/ssGSEA2.0.R",
             "/mnt/c/Users/redas/Desktop/jupyter_directory/salomon_lab_folder/rdn_soq_project/proteomics/database/ptm.sig.db.all.flanking.human.v2.0.0.gmt")


In [ ]:
# Filter out all the nonsense from the PTM-SEA output, keep
# only the enrichment scores, q-values, and groups
mso.imp_main("/mnt/c/Users/redas/Desktop/jupyter_directory/salomon_lab_folder/rdn_soq_project/proteomics",
             "output-combined.gct",
             "\t",
             "output_combined_heatmap.txt")

In [ ]:
# Make the bubbleplots :)
enrich_bubbleplot_list([f for f in ptm_files if "time" in f],
                       [d for d in outdirs if "time" in d],
                       sig_exception = ["ween"],
                       significance = 0.2,
                       group_heads = ["D 2.5 min", "D 10 min",
                                      "R 2.5 min", "R 10 min",
                                      "S 2.5 min", "S 10 min",],
                       max_score = 6)

enrich_bubbleplot_list([f for f in ptm_files if "cont" in f],
                       [d for d in outdirs if "cont" in d],
                       sig_exception = ["ween"],
                       significance = 0.2,
                       group_heads = ["R 0 min", "R 2.5 min", "R 10 min",
                                      "S 0 min", "S 2.5 min", "S 10 min"],
                       max_score = 6)

In [ ]:
help(gh)